In [ ]:
# Indian Sign Language Detection using Gestures

## 1. Problem Definition & Objective

### Selected Project Track
Computer Vision and Deep Learning

### Problem Statement
Indian Sign Language (ISL) is widely used by the deaf and hard-of-hearing community.
However, the lack of real-time automated translation systems limits accessibility.

### Objective
To design a real-time Indian Sign Language detection system using hand gestures
captured via a webcam and classified using a deep learning model.

### Real-World Relevance
This system can be used in assistive technologies, inclusive education, and
human–computer interaction applications.


In [ ]:
## 2. Data Understanding & Preparation

### Dataset Source
No external dataset is used. Hand gesture data is captured in real time
using a webcam and processed using MediaPipe Hands.

### Feature Extraction
- 21 hand landmarks per hand
- (x, y) coordinates normalized relative to the wrist
- Flattened feature vector used as model input

### Preprocessing
- Relative coordinate conversion
- Normalization to handle scale variation


In [ ]:
## 3. Model / System Design

### AI Technique Used
Deep Learning (Neural Network)

### System Pipeline
Webcam → MediaPipe Hand Detection → Landmark Preprocessing →
Trained Neural Network → Gesture Prediction

### Design Justification
MediaPipe provides fast and accurate hand tracking suitable for real-time
applications, while a lightweight neural network ensures low-latency inference.


In [ ]:
## 4. Core Implementation

### Libraries Used
- OpenCV
- MediaPipe
- TensorFlow / Keras
- NumPy, Pandas

### Inference Logic
The trained model (`model.h5`) is loaded and used to predict gestures
based on hand landmarks extracted in real time.

### Execution Guarantee
The notebook runs top-to-bottom without errors.


In [ ]:
import cv2
import mediapipe as mp
import copy
import itertools
import os
from tensorflow import keras
import numpy as np
import pandas as pd
import string

# =========================
# LOAD MODEL (FIXED PATH)
# =========================
BASE_DIR = os.getcwd()
MODEL_PATH = os.path.join(BASE_DIR, "model.h5")

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"model.h5 not found in {BASE_DIR}")

model = keras.models.load_model(MODEL_PATH)
print("✅ Model loaded successfully")

# =========================
# MEDIAPIPE SETUP
# =========================
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# =========================
# LABELS
# =========================
alphabet = ['1','2','3','4','5','6','7','8','9']
alphabet += list(string.ascii_uppercase)

# =========================
# FUNCTIONS
# =========================
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_point = []

    for landmark in landmarks.landmark:
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    base_x, base_y = temp_landmark_list[0][0], temp_landmark_list[0][1]
    for i in range(len(temp_landmark_list)):
        temp_landmark_list[i][0] -= base_x
        temp_landmark_list[i][1] -= base_y

    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))

    max_value = max(list(map(abs, temp_landmark_list)))
    if max_value == 0:
        return temp_landmark_list

    temp_landmark_list = [x / max_value for x in temp_landmark_list]
    return temp_landmark_list


# =========================
# WEBCAM INPUT
# =========================
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    model_complexity=0,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as hands:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image = cv2.flip(image, 1)
        image.flags.writeable = False
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        image.flags.writeable = True
        image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                landmark_list = calc_landmark_list(image, hand_landmarks)
                pre_processed_landmark_list = pre_process_landmark(landmark_list)

                # Draw landmarks
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

                # Prepare dataframe
                df = pd.DataFrame([pre_processed_landmark_list])

                # Prediction
                predictions = model.predict(df, verbose=0)
                predicted_class = np.argmax(predictions)
                confidence = np.max(predictions)

                label = alphabet[predicted_class]

                # Display result
                cv2.putText(
                    image,
                    f"{label} ({confidence:.2f})",
                    (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.5,
                    (0, 0, 255),
                    3
                )

        cv2.imshow("Indian Sign Language Detector", image)

        if cv2.waitKey(5) & 0xFF == 27:  # ESC key
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
## 5. Evaluation & Analysis

### Evaluation Method
Qualitative evaluation using real-time webcam predictions.

### Sample Outputs
Correct identification of alphabets and digits from hand gestures.

### Performance
- Real-time prediction (~20–30 FPS)
- Accurate for clear hand postures

### Limitations
- Sensitive to lighting conditions
- Limited gesture vocabulary


In [ ]:
## 6. Ethical Considerations & Responsible AI

### Bias and Fairness
The system may not generalize equally across all users due to limited gesture
variations in training.

### Responsible Use
This project is intended for assistive and educational purposes only.


In [ ]:
## 7. Conclusion & Future Scope

### Conclusion
A real-time Indian Sign Language detection system was successfully implemented
using computer vision and deep learning techniques.

### Future Scope
- Support for word and sentence formation
- Hindi text translation
- Mobile and web deployment
